# stats for all the spotify-web-api track-feature json files in local SONG_DATA folder

# https://developer.spotify.com/documentation/web-api/reference/get-audio-features

### <span style="color:orange">danceability</span>
Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
### <span style="color:orange">energy</span>
Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
### <span style="color:orange">key</span>
The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.
### <span style="color:orange">loudness</span>
The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.
### <span style="color:orange">mode</span>
Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.
### <span style="color:orange">speechiness</span>
Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
### <span style="color:orange">acousticness</span>
A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
### <span style="color:orange">instrumentalness</span>
Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
### <span style="color:orange">liveness</span>
Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.
### <span style="color:orange">valence</span>
A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).
### <span style="color:orange">tempo</span>
The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
### <span style="color:orange">type</span>
The object type.
### <span style="color:orange">id</span>
The Spotify ID for the track.
### <span style="color:orange">uri</span>
The Spotify URI for the track.
### <span style="color:orange">track_href</span>
A link to the Web API endpoint providing full details of the track.
### <span style="color:orange">analysis_url</span>
A URL to access the full audio analysis of this track. An access token is required to access this data.
### <span style="color:orange">duration_ms</span>
The duration of the track in milliseconds.
### <span style="color:orange">time_signature</span>
An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4".
"""

In [21]:
import os
import json
import pandas as pd

def show_statistics(folder_path):
    file_names = [f for f in os.listdir(folder_path) if f.endswith('.json') and '_features' in f]
    all_data = []

    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # Extracting only the desired keys from the data
        extracted_data = {key: data[key] for key in [
            'danceability', 'energy', 'key', 'loudness', 'mode', 
            'speechiness', 'acousticness', 'instrumentalness', 
            'liveness', 'valence', 'tempo', 'time_signature']}
        
        all_data.append(extracted_data)

    # Convert the list of dictionaries to DataFrame
    all_data = pd.DataFrame(all_data)
    
    # Ensure the values are numeric
    all_data = all_data.apply(pd.to_numeric, errors='coerce')
    
    # Drop rows with NaN values if any
    all_data = all_data.dropna()
    
    stats = all_data.describe().transpose()[['mean', 'min', 'max']]
    stats['median'] = all_data.median()
    
    # Rearrange the columns to be in the order: mean, median, max, min
    stats = stats[['mean', 'median', 'max', 'min']]
    
    print(stats)

# Example usage
# Pass the folder path as an argument to the function
show_statistics('/Users/walkerhutchinson/Desktop/SPOTIFY_STUFF/SIGILCORE_DATA')


                        mean      median       max        min
danceability        0.502833    0.502000    0.5940   0.378000
energy              0.853083    0.858000    0.9810   0.565000
key                 5.916667    7.000000   11.0000   1.000000
loudness           -5.192333   -5.096500   -0.9860  -9.437000
mode                0.666667    1.000000    1.0000   0.000000
speechiness         0.186608    0.137500    0.4340   0.033400
acousticness        0.014436    0.009060    0.0566   0.000043
instrumentalness    0.045969    0.000024    0.5500   0.000000
liveness            0.239800    0.186500    0.5410   0.065000
valence             0.447825    0.504500    0.7520   0.039900
tempo             133.256250  142.289500  151.9280  75.134000
time_signature      4.000000    4.000000    4.0000   4.000000


## adjusted for outliers

In [22]:
import os
import json
import pandas as pd

def remove_outliers(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    df_out = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df_out

def show_statistics(folder_path):
    file_names = [f for f in os.listdir(folder_path) if f.endswith('.json') and '_features' in f]
    all_data = []

    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # Extracting only the desired keys from the data
        extracted_data = {key: data[key] for key in [
            'danceability', 'energy', 'key', 'loudness', 'mode', 
            'speechiness', 'acousticness', 'instrumentalness', 
            'liveness', 'valence', 'tempo', 'time_signature']}
        
        all_data.append(extracted_data)

    # Convert the list of dictionaries to DataFrame
    all_data = pd.DataFrame(all_data)
    
    # Ensure the values are numeric
    all_data = all_data.apply(pd.to_numeric, errors='coerce')
    
    # Drop rows with NaN values if any
    all_data = all_data.dropna()
    
    # Remove outliers
    all_data = remove_outliers(all_data)
    
    stats = all_data.describe().transpose()[['mean', 'min', 'max']]
    stats['median'] = all_data.median()
    
    # Rearrange the columns to be in the order: mean, median, max, min
    stats = stats[['mean', 'median', 'max', 'min']]
    
    print(stats)

# Example usage
# Pass the folder path as an argument to the function
show_statistics('/Users/walkerhutchinson/Desktop/SPOTIFY_STUFF/SIGILCORE_DATA')

                        mean      median         max         min
danceability        0.512000    0.509500    0.594000    0.378000
energy              0.886875    0.858000    0.981000    0.824000
key                 6.125000    7.000000   11.000000    1.000000
loudness           -4.823500   -4.564500   -0.986000   -9.437000
mode                0.750000    1.000000    1.000000    0.000000
speechiness         0.212638    0.212500    0.434000    0.052200
acousticness        0.018480    0.017600    0.056600    0.000043
instrumentalness    0.000065    0.000024    0.000272    0.000000
liveness            0.238325    0.177500    0.541000    0.077600
valence             0.499250    0.551000    0.752000    0.146000
tempo             142.314500  142.289500  151.657000  132.074000
time_signature      4.000000    4.000000    4.000000    4.000000


# print-to-terminal version below

In [1]:
import os
import json
import numpy as np

def print_statistics(folder_path):
    # List all the JSON files in the given folder path with "_features" in the title
    file_names = [f for f in os.listdir(folder_path) if f.endswith('.json') and '_features' in f]

    # Create a dictionary to store all the features and their values
    all_data = { 
        'danceability': [],
        'energy': [],
        'key': [],
        'loudness': [],
        'mode': [],
        'speechiness': [],
        'acousticness': [],
        'instrumentalness': [],
        'liveness': [],
        'valence': [],
        'tempo': []
    }

    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)
        
        # Load the JSON data from the file
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # Append the data to all_data dictionary
        for feature in all_data.keys():
            all_data[feature].append(data[feature])
    
    # Compute and print the statistics for each feature
    for feature, values in all_data.items():
        values = np.array(values)
        mean_value = np.mean(values)
        min_value = np.min(values)
        max_value = np.max(values)

        print(f'Statistics for {feature}:')
        print(f'    Mean: {mean_value:.2f}')
        print(f'    Min: {min_value:.2f}')
        print(f'    Max: {max_value:.2f}')
        print('')

# Example usage
# Pass the folder path as argument to the function
print_statistics('/Users/walkerhutchinson/Desktop/SPOTIFY_STUFF/SONG_DATA')


Statistics for danceability:
    Mean: 0.68
    Min: 0.27
    Max: 0.95

Statistics for energy:
    Mean: 0.70
    Min: 0.33
    Max: 0.98

Statistics for key:
    Mean: 5.08
    Min: 0.00
    Max: 11.00

Statistics for loudness:
    Mean: -6.24
    Min: -11.68
    Max: -0.99

Statistics for mode:
    Mean: 0.68
    Min: 0.00
    Max: 1.00

Statistics for speechiness:
    Mean: 0.24
    Min: 0.03
    Max: 0.66

Statistics for acousticness:
    Mean: 0.08
    Min: 0.00
    Max: 0.63

Statistics for instrumentalness:
    Mean: 0.02
    Min: 0.00
    Max: 0.55

Statistics for liveness:
    Mean: 0.22
    Min: 0.06
    Max: 0.72

Statistics for valence:
    Mean: 0.50
    Min: 0.04
    Max: 0.96

Statistics for tempo:
    Mean: 129.20
    Min: 74.52
    Max: 186.04

